# TIC-TAC-TOE Markov Decision Process

In [1]:
import numpy as np

## Auxiliary functions to build the model

In [2]:
def board_to_id(B):
    """Takes a board matrix and returns a base-10 board ID."""
    symbols = B.flatten()
    id = 0
    for i in range(9):
        id += int(symbols[i] * (3 ** (8 - i)))
    return id

def id_to_board(id):
    """Takes a base-10 board ID and returns a board matrix."""
    board_str = np.base_repr(id, base=3).zfill(9)
    return np.array(list(board_str), dtype=np.int8).reshape(3, 3)